In [108]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D
from keras.layers import Dropout, Activation, Conv1D, GlobalMaxPooling1D

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
from termcolor import colored



In [109]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [110]:

# Import twitter data
print(colored("Loading train and test data", "green"))

tr = pd.read_csv("/content/drive/My Drive/big.csv")
te = pd.read_csv("/content/drive/My Drive/twt_#ifc.csv")

tr.full_text=tr.full_text.astype(str)
te.full_text=te.full_text.astype(str)
print(colored("... \nData loaded", "green"))


Loading train and test data
... 
Data loaded


In [111]:

# Tokenization
print(colored("Tokenizing and padding data", "green"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(tr['full_text'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(tr['full_text'].astype(str).values)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(te['full_text'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))

Tokenizing and padding data
Tokenizing and padding complete


In [121]:
# Building the model
print(colored("Creating the 1 Layer CNN model", "yellow"))
model_prediction = Sequential()
model_prediction.add(Embedding(10000, 50, input_length=37))
model_prediction.add(Dropout(0.5))
model_prediction.add(Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1))
model_prediction.add(GlobalMaxPooling1D())
model_prediction.add(Dense(250))
model_prediction.add(Activation('relu'))
model_prediction.add(Dense(3))
model_prediction.add(Activation('sigmoid'))
model_prediction.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   

model_prediction.summary()

Creating the 1 Layer CNN model
Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_45 (Embedding)     (None, 37, 50)            500000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 37, 50)            0         
_________________________________________________________________
conv1d_220 (Conv1D)          (None, 35, 250)           37750     
_________________________________________________________________
global_max_pooling1d_19 (Glo (None, 250)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 250)               62750     
_________________________________________________________________
activation_19 (Activation)   (None, 250)               0         
_________________________________________________________________
dense_45 (Dense)      

In [122]:
# Training the model
print(colored("Training the 1 Layer CNN model", "green"))
history = model_prediction.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the 1 Layer CNN model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 6s 302us/step - loss: 0.5299 - accuracy: 0.7131
Epoch 2/3
20741/20741 [==============================] - 6s 293us/step - loss: 0.4041 - accuracy: 0.8099
Epoch 3/3
20741/20741 [==============================] - 6s 296us/step - loss: 0.3598 - accuracy: 0.8372


In [123]:
 print(colored("Testing the 1 Layer CNN model", "green"))

loss, accuracy = model_prediction.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x=accuracy
score, accuracy = model_prediction.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))

y=accuracy

Testing the 1 Layer CNN model
20741/20741 [==============================] - 1s 61us/step
Train accuracy: 0.866560697555542
1749/1749 [==============================] - 0s 63us/step
Test accuracy: 0.8509624004364014


In [39]:
# Building the model
print(colored("Creating the 9 Layer CNN model", "yellow"))
model9_prediction = Sequential()
model9_prediction.add(Embedding(10000, 50, input_length=train_tweets.shape[1]))
model9_prediction.add(Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1))
model9_prediction.add(Conv1D(filters=250, kernel_size=3))
model9_prediction.add(Conv1D(filters=250, kernel_size=3))
model9_prediction.add(Conv1D(filters=250, kernel_size=3))
model9_prediction.add(Conv1D(filters=250, kernel_size=3))
model9_prediction.add(Conv1D(filters=250, kernel_size=3))
model9_prediction.add(Conv1D(filters=250, kernel_size=3))
model9_prediction.add(Dense(3))
model9_prediction.add(GlobalMaxPooling1D())
model9_prediction.add(Dense(3))
model9_prediction.add(Activation('relu'))
model9_prediction.add(Dense(3))
model9_prediction.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   

model9_prediction.summary()

Creating the 9 Layer CNN model
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 37, 50)            500000    
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 35, 250)           37750     
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 33, 250)           187750    
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 31, 250)           187750    
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 29, 250)           187750    
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 27, 250)           187750    
_________________________________________________________________
conv1d_16 (Conv1D)    

In [40]:
# Training the model
print(colored("Training the 9 Layer CNN model", "green"))
history = model9_prediction.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the 9 Layer CNN model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 124s 6ms/step - loss: 1.0824 - accuracy: 0.6336
Epoch 2/3
20741/20741 [==============================] - 117s 6ms/step - loss: 1.0028 - accuracy: 0.6667
Epoch 3/3
20741/20741 [==============================] - 117s 6ms/step - loss: 0.9927 - accuracy: 0.6848


In [100]:
 print(colored("Testing the 9 Layer CNN model", "green"))

loss, accuracy = model9_prediction.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x1=accuracy
score, accuracy = model9_prediction.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y1=accuracy



Testing the 9 Layer CNN model
20741/20741 [==============================] - 17s 798us/step
Train accuracy: 0.7031642198562622
1749/1749 [==============================] - 1s 800us/step
Test accuracy: 0.7061178088188171


In [71]:
from keras.layers import Convolution2D,Reshape

In [74]:
# Building the model
print(colored("Creating the 19 Layer CNN model", "yellow"))
model19_prediction = Sequential()
model19_prediction.add(Embedding(10000, 50, input_length=train_tweets.shape[1]))
model19_prediction.add(Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1))

model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))

model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))

model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))
model19_prediction.add(Conv1D(filters=250, kernel_size=3))


model19_prediction.add(Dense(3))
model19_prediction.add(GlobalMaxPooling1D())
model19_prediction.add(Dense(3))
model19_prediction.add(Activation('relu'))
model19_prediction.add(Dense(3))
model19_prediction.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   

model19_prediction.summary()

Creating the 19 Layer CNN model
Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_38 (Embedding)     (None, 37, 50)            500000    
_________________________________________________________________
conv1d_198 (Conv1D)          (None, 35, 250)           37750     
_________________________________________________________________
conv1d_199 (Conv1D)          (None, 33, 250)           187750    
_________________________________________________________________
conv1d_200 (Conv1D)          (None, 31, 250)           187750    
_________________________________________________________________
conv1d_201 (Conv1D)          (None, 29, 250)           187750    
_________________________________________________________________
conv1d_202 (Conv1D)          (None, 27, 250)           187750    
_________________________________________________________________
conv1d_203 (Conv1D)  

In [90]:
# Training the model
print(colored("Training the 19 Layer CNN model", "green"))
history = model19_prediction.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 1, batch_size = 128)
print(colored(history, "green"))

Training the 19 Layer CNN model
Epoch 1/1
20741/20741 [==============================] - 248s 12ms/step - loss: 5.1404 - accuracy: 0.0000e+00


In [98]:
 print(colored("Testing the 19 Layer CNN model", "green"))

loss, accuracy =model19_prediction.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))

x2=accuracy
score, accuracy = model19_prediction.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))

y2=accuracy


Testing the 19 Layer CNN model
20741/20741 [==============================] - 27s 1ms/step
Train accuracy: 0.0
1749/1749 [==============================] - 2s 1ms/step
Test accuracy: 0.0


5.1404102305989845 0.0


In [84]:
from tabulate import tabulate


In [125]:
print(tabulate([
    ['1 Layer CNN',x,y], 
    ['9 Layer CNN',x1,y1],
    ['19 Layer CNN',x2,y2]], 
    
    headers=['Model', 'Training Accuracy','Testing Accuracy']))

Model           Training Accuracy    Testing Accuracy
------------  -------------------  ------------------
1 Layer CNN              0.866561            0.850962
9 Layer CNN              0.703164            0.706118
19 Layer CNN             0                   0
